In [14]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

In [15]:
tmp <- subset(df, select = -c(deg_cat))           #hot encoding

#define one-hot encoding function
dummy <- dummyVars(" ~ .", data=tmp)

#perform one-hot encoding on data frame
final_df <- data.frame(predict(dummy, newdata=tmp))


final_df$deg_cat <- df$deg_cat


df <- final_df

## Data partition

In [16]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)


TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [17]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    370     370 

## Models

In [18]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [19]:

model_monmlp <- train(x=TrainData, y=TrainClasses, 
               method = "monmlp") 

** Ensemble 1 
Complex eigenvalues found for method = BFGS 
coefficients for function value 0.04229625  :
  [1]  5.284568e+01 -1.626574e+02  1.918574e+01 -1.487039e+02 -1.137012e+02
  [6]  1.755895e+02  1.576402e+02 -3.224142e+01  3.219735e+01 -5.098731e+01
 [11]  5.860459e+01 -1.957733e+01 -8.911614e-01  8.831790e+00  1.137972e+01
 [16] -1.142318e+01 -4.938587e+01 -3.157243e+01 -5.342604e+01  8.377201e+01
 [21]  6.993147e+00 -6.952341e+00  4.585600e+01 -1.468444e+01 -4.721692e+01
 [26]  4.873459e+00 -3.800375e+00 -6.190346e+00  4.050106e+00  4.043578e+01
 [31] -1.143084e+02  9.498848e+01 -2.250763e+01  1.465157e+01 -1.468310e+01
 [36]  1.301658e+01 -1.301988e+01 -3.719029e+00  3.788592e+00 -6.460813e+01
 [41]  8.990025e+00  1.631531e+01 -3.425124e+00  8.865294e+01 -4.936240e+01
 [46]  4.935867e+01  1.970659e+01 -8.777544e+01 -1.379253e+02 -5.055662e+01
 [51]  1.078325e+02  4.263985e+01 -2.031909e+01  3.642413e+01  4.232962e+01
 [56]  2.824729e+01  1.036232e+01  7.562628e+01 -7.674621e

In [20]:

model_mlpW <- train(x=TrainData, y=TrainClasses, 
               method = "mlpWeightDecay") 

## Results

In [21]:
print(model_monmlp)
plot(model_monmlp,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Monotone Multi-Layer Perceptron Neural Network 

740 samples
183 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 740, 740, 740, 740, 740, 740, ... 
Resampling results across tuning parameters:

  hidden1  Accuracy   Kappa    
  1        0.8738621  0.7475803
  3        0.8678767  0.7357358
  5        0.8642923  0.7282586

Tuning parameter 'n.ensemble' was held constant at a value of 1
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were hidden1 = 1 and n.ensemble = 1.


In [22]:
print(model_mlpW)
plot(model_mlpW,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Multi-Layer Perceptron 

740 samples
183 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 740, 740, 740, 740, 740, 740, ... 
Resampling results across tuning parameters:

  size  decay  Accuracy   Kappa    
  1     0e+00  0.8772877  0.7538641
  1     1e-04  0.8561920  0.7117120
  1     1e-01  0.5054723  0.0000000
  3     0e+00  0.8915686  0.7827969
  3     1e-04  0.8808614  0.7612791
  3     1e-01  0.5038956  0.0000000
  5     0e+00  0.8936837  0.7868182
  5     1e-04  0.8847307  0.7687902
  5     1e-01  0.4988498  0.0000000

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were size = 5 and decay = 0.


In [23]:
# collect resamples
results <- resamples(list(monmlp=model_monmlp,  mlpWeightDecay=model_mlpW))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: monmlp, mlpWeightDecay 
Number of resamples: 25 

Accuracy 
                    Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
monmlp         0.8222222 0.8586957 0.8736842 0.8738621 0.8825623 0.9052632    0
mlpWeightDecay 0.8478261 0.8782288 0.8965517 0.8936837 0.9018868 0.9343066    0

Kappa 
                    Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
monmlp         0.6444444 0.7189850 0.7469291 0.7475803 0.7649608 0.8117431    0
mlpWeightDecay 0.6976841 0.7577932 0.7928754 0.7868182 0.8016080 0.8681565    0


## Prediction

In [24]:
pred_mlpW <- predict(model_mlpW,TestData)
pred_mlpW.prob <- predict(model_mlpW,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_mlpW) 
result
result$byClass

Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      22      20
   NotFast   43     233
                                          
               Accuracy : 0.8019          
                 95% CI : (0.7538, 0.8443)
    No Information Rate : 0.7956          
    P-Value [Acc > NIR] : 0.422555        
                                          
                  Kappa : 0.2987          
                                          
 Mcnemar's Test P-Value : 0.005576        
                                          
            Sensitivity : 0.33846         
            Specificity : 0.92095         
         Pos Pred Value : 0.52381         
         Neg Pred Value : 0.84420         
             Prevalence : 0.20440         
         Detection Rate : 0.06918         
   Detection Prevalence : 0.13208         
      Balanced Accuracy : 0.62971         
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
          0.33846154           0.92094862           0.52380952 
      Neg Pred Value            Precision               Recall 
          0.84420290           0.52380952           0.33846154 
                  F1           Prevalence       Detection Rate 
          0.41121495           0.20440252           0.06918239 
Detection Prevalence    Balanced Accuracy 
          0.13207547           0.62970508

In [25]:
roc_knn <- multiclass.roc(TestClasses, as.numeric(pred_mlpW))
roc_knn


Setting direction: controls < cases




Call:
multiclass.roc.default(response = TestClasses, predictor = as.numeric(pred_mlpW))

Data: as.numeric(pred_mlpW) with 2 levels of TestClasses: fast, NotFast.
Multi-class area under the curve: 0.684

In [26]:
roc_obj

ERROR: Error in eval(expr, envir, enclos): oggetto "roc_obj" non trovato


In [ ]:
pred_svmMlpW.prob